In [5]:
from joblib import load, dump
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from scikeras.wrappers import KerasClassifier
import optuna

combined_data = load('../../DataDumps/all_data_fft - balanced YesandNo.joblib')
combined_labels = load('../../DataDumps/2class_labels.joblib')


X_train_val, X_test, y_train_val, y_test = train_test_split(combined_data, combined_labels, test_size=0.2, random_state=42)

input_sh = X_train_val.shape

In [6]:
def create_model():
    model = Sequential()
    model.add(Conv1D(8, kernel_size=9, strides=1, activation="relu", input_shape=input_sh))

    model.add(Conv1D(8, kernel_size=7, strides=1, activation="relu"))

    model.add(Conv1D(8, kernel_size=7, strides=1, activation="relu"))

    model.add(Conv1D(16, kernel_size=7, strides=1, activation="relu"))

    model.add(Conv1D(16, kernel_size=5, strides=1, activation="relu"))

    model.add(Conv1D(16, kernel_size=5, strides=1, activation="relu"))

    model.add(Conv1D(32, kernel_size=5, strides=2, activation="relu"))

    model.add(Conv1D(32, kernel_size=3, strides=1, activation="relu"))

    model.add(Conv1D(32, kernel_size=3, strides=1, activation="relu"))

    
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   
    return model


In [7]:
model = create_model()

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv1d_9. Consider increasing the input size. Received input shape [None, 1659, 3, 7201] which would produce output shape with a zero or negative value in a dimension.

In [8]:
model.fit(X_train_val, y_train_val)

ValueError: in user code:

    File "c:\Users\Etienne\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Etienne\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Etienne\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Etienne\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Etienne\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Etienne\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compat

In [ ]:


# Optuna Objective Function
def objective(trial):
    lstm_2 = 0
    drop_2 = 0
    lstm_3 = 0
    drop_3 = 0
    num_lstm = trial.suggest_int('num_lstm_layers', 1, 3)
    lstm_1 = trial.suggest_int('lstm_units_1', 32, 128)
    dropout_1 = trial.suggest_float('dropout_1', 0.0, 0.5)
    optimizer = trial.suggest_categorical('optimizer', ['SGD', 'RMSprop', 'Adam'])

    if num_lstm > 1:
        lstm_2 = trial.suggest_int(f'lstm_units_2', 32, 128)
        drop_2 = trial.suggest_float(f'dropout_2', 0.0, 0.5)
    if num_lstm > 2:
        lstm_3 = trial.suggest_int(f'lstm_units_3', 32, 128)
        drop_3 = trial.suggest_float(f'dropout_3', 0.0, 0.5)


    # Assuming StratifiedKFold, customize if needed
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)  
    scores = []
    for train_index, test_index in skf.split(X_train_val, y_train_val):
        X_tr, X_val = X_train_val[train_index], X_train_val[test_index]
        y_tr, y_val = y_train_val[train_index], y_train_val[test_index]

        model = create_model(optimizer, num_lstm, lstm_1, dropout_1, lstm_2, drop_2, lstm_3, drop_3)
        model.fit(X_tr, y_tr, verbose = 0) 
        score = model.evaluate(X_val, y_val, verbose = 0)
        scores.append(score[1])
    return np.array(scores).mean()

# Hyperparameter Optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

best_params = study.best_params
print("Best hyperparameters:", best_params)